<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction


## How to use this notebook
This example runs Sima in the cloud based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. This notebook assumes that all tooling is in place see the installation section of the notebook for [local execution](workflowCoreDemoSE28SestraAndWasim.ipynb#installation)

This note bookshould be run in the following way:
1. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
2. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
3. [Run analysis](#run) shall be run each time a new analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.


# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Install or upgrade needed Python packages
Run only when packages needs to be upgraded

In [1]:
update_packages = False
if update_packages:
    ! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-onecompute
    ! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-oneworkflow
    ! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-sesam-commands

## Set up fixed user parameters <a id='fixed'></a>
The below parameters should only be needed to modify once.

In [2]:
#Sima path must be specified
sima_exe_path = r'C:\Program Files\DNV\Sima V4.4-00'         
tempFolderPath = r"c:\OneWorkflowTemp" #local directory for worker host, should be at root due to issues with long file names on Windows


## Set up custom user parameters <a id='custom'></a>

In [3]:
# local workspace, all results will be put here after local or cloud runs
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud'
# location of common files for all analysis, has to be below workspacePath
workspaceId = "SE28"
loadcase_file = "test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = True


## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [4]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tempFolderPath)
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workflow_client.common_directory
results_folder = workflow_client.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


c:\OneWorkflowTemp


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [5]:
from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\copyFiles.py to SE28/CommonFiles/copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\Floater.ssg to SE28/CommonFiles/Floater.ssg
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\sestra_template.inp to SE28/CommonFiles/sestra_template.inp
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\SimaTemplate.stask to SE28/CommonFiles/SimaTemplate.stask
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\T3.FEM to SE28/CommonFiles/T3.FEM
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\test.py to SE28/CommonFiles/test.py
Uploading C:\Users\kblu\source\repos\i

## Prepare Sima input  <a id='prepare'></a>


### Create command line input for Sima
The function below will create the command line input that will be used to run Sima. 

In [6]:
from dnv.onecompute import FileSpecification


def get_commands_inputs(
        stask_file: str,
        case: dict[str, Any])->dict[str, dict[str, Any]]:
    commands = dict[str, Any]()
    commands["--consoleLog"] = ""
    commands["--log-level"] = "ALL"
    commands["--data"] = "."
    commands["--import"] = dict(file=FileSpecification(sharedfolder=True,
                                directory=commonfiles_folder, filename=stask_file))
    commands["--run"] = dict(task="WorkflowTask",
                             workflow="ExampleWorkflow")

    return {"commands": commands, "inputs": case}

### Create load cases with unique Sima input <a id='loadcase_id'></a>
This code generates the input needed for Sima for the individual load cases. The input parameters to vary in the Sima run are give in the loadcasefile.

In [7]:
import pandas as pd
from dnv.sesam.sima_command import SimaCommand
from dnv.onecompute.flowmodel import ParallelWork
from dnv.oneworkflow import PythonCommand
def get_parallel_work(single_task: bool = False):
    """Returns a parallel processing unit. If the single_task parameter is set to True, the unit will only contain the first task. 
    If set to False, all tasks will be included. The single task option is primarily used for testing purposes."""
    os.chdir(workspace.workspace_path)
    load_cases_parent_folder_name = workspace.load_cases_parent_directory

    parallel_work = ParallelWork()
    parallel_work.work_items.clear()

    # Open environmental input file
    df_cases = pd.read_excel(os.path.join(workspacePath, loadcase_file), index_col=0)
    for loadcase_folder_name, case in df_cases.iterrows():
        load_case_folder = os.path.join(
            load_cases_parent_folder_name, loadcase_folder_name)
        result_folder_lc = os.path.join(workspace.results_directory, loadcase_folder_name)
        # Get SIMA commands and inputs 
        commands_inputs = get_commands_inputs(
             stask_file,  case.to_dict()
        )
        test_command = PythonCommand(
            directory=commonfiles_folder,
            filename="test.py",
            working_dir=loadcase_folder_name,
        )
        # Create SimaCommand instance
        sima_cmd = SimaCommand(sima_exe_path)
        sima_cmd.commands = commands_inputs["commands"]
        sima_cmd.input = commands_inputs["inputs"]
        sima_cmd.sima_result_files =[
                "*-sima.lis",
                "variable*.inp",
                "*.log",
                "results.tda",
                "results.txt",
                "sima_*.res",
                "sys-sima.dat",
                "sima_*.bin",
                "key_sima_*.txt",
                "sima.*"
            ]
        sima_cmd.working_directory = load_case_folder

        # Add work item to ParallelWork instance
        parallel_work.add(sima_cmd, work_unit_id=loadcase_folder_name).output_directory(result_folder_lc,
                                                                                              include_files=["**/sima.*","**/*.txt", "**/*.lis", "**/*.log"])
        if single_task == True:
            break
    return parallel_work

# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [8]:
from oneWorkflowToolBox import run_workflow_async
import json

"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

# Create Parallel Work Unit and Job
work_unit = get_parallel_work()
job = workflow_client.create_job(work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg"])
await run_workflow_async(job, workflow_client, downloadOptions)


Error: load cases directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\LoadCases does not exist.
The work item 3068c97c-7d5b-481a-adeb-af1b2772c909 message is 'Pending'
The work item test001 message is 'Pending'
The work item test002 message is 'Pending'
The work item test003 message is 'Pending'
The progress of the job is '0%'. The message is ''
The work item test001 message is ''
The work item test002 message is ''
The work item test001 message is '2023-05-30 12:27:10,041 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test002 message is '2023-05-30 12:27:09,418 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test001 message is '2023-05-30 12:27:32,757 [main] DEBUG GrimReaper - /mnt/batch/tasks/workitems/159fe760-9ac6-42f5-b504-9f6cbd5789d0/job-1/test001/wd/LoadCases/test001/.tmp/

# Run Wasim and Sestra <a id='runwasim'></a>

In [9]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.sesam.sestra_command import SestraCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
import json
import os
workspace.results_directory = "Results"
load_cases = ["test001", "test002", "test003"]
workflow_windows_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tempFolderPath, Platform.Windows)
os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue

        casedict = case.to_dict()
       
        load_case_result_files_dir = os.path.join(results_directory, casename)
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd,  f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"
                                                                         ,"**/*.LIS", "**/*.MLG"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)


workflow_windows_client.login()
work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)

downloadOptions = FileOptions(
    #max_size_bytes=1001124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
await workflow_windows_client.run_workflow_async(
    work_unit,
    work_item_status_changed=work_item_status_changed_callback(
        workflow_windows_client, downloadOptions),
    log_job=True
)


c:\OneWorkflowTemp
{
    "$type": "DNVGL.One.Compute.Core.FlowModel.Job, DNVGL.One.Compute.Core",
    "properties": {},
    "JobId": "13f48ddc-ea4d-4dad-b7ac-d493d2bba76b",
    "ServiceName": "ImproveFlowWorker",
    "PoolId": "",
    "JobPreparationWork": {
        "$type": "DNVGL.One.Compute.Core.FlowModel.WorkUnit, DNVGL.One.Compute.Core",
        "properties": {},
        "id": "8247aed6-3724-4a34-b4ce-7e285616f2ec",
        "TaskRole": 2,
        "Data": {
            "Version": 1
        },
        "Command": "OneWorkflowWorkerHost",
        "InputFileSpecifications": [
            {
                "$type": "DNVGL.One.Compute.Core.FlowModel.DataTransfer.FileTransferSpecification, DNVGL.One.Compute.Core",
                "SourceSpecification": {
                    "$type": "DNVGL.One.Compute.Core.FlowModel.DataTransfer.BlobDirectorySpecification, DNVGL.One.Compute.Core",
                    "Directory": "SE28\\CommonFiles",
                    "ContainerUrl": "https://tstpeuwst0